# OFDM-GAN-SR Training on Google Colab

This notebook trains the CWGAN-GP model for OFDM signal reconstruction.

In [ ]:
# Clone repository
!git clone https://github.com/orpheus016/ofdm-gan-sr.git
%cd ofdm-gan-sr

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Start training (skip export so we can rerun export safely)
!python train.py --synthetic --epochs 100 --lr 0.0001 --batch_size 16 --skip_export

In [ ]:
# Optional: Monitor training with TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs/

## Export weights only (safe to rerun)
After training (or resuming a previous run), export weights from a saved checkpoint without retraining.

In [ ]:
# Optional: Save checkpoints to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create backup directory
!mkdir -p /content/drive/MyDrive/ofdm-gan-checkpoints

# Copy checkpoints (run this after training completes or periodically)
!cp -r checkpoints/* /content/drive/MyDrive/ofdm-gan-checkpoints/
!cp -r runs/* /content/drive/MyDrive/ofdm-gan-checkpoints/runs/

print("✓ Checkpoints saved to Google Drive")

In [ ]:
# Export from final checkpoint (adjust path if needed)
!python utils/export.py --checkpoint checkpoints/final_model.pt --export_dir ./export